Assignment 21

In [65]:
import numpy as np
import cv2
import math
import sys
import time
import enum

class Player:
    def __init__(self, id, pos, score = 0):
        self.id = id
        self.score = score
        self.position = pos
        self.universesWon = np.uint64(0)

def PlayTillEndDeterministic(players):
    diceRoll = 0
    
    index = 0
    while(True):
        player = players[index]
        print("current position: {}".format(player.position))
        print("current score: {}".format(player.score))
        for k in range (3):
            diceValue = 0
            diceRoll +=1
            if diceRoll % 100 == 0:
                diceValue = 100
            else:
                diceValue = diceRoll % 100
            print("Player {} rolled {}, diceRoll: {}".format(player.id, diceValue, diceRoll))
            
            
            player.position += diceValue 
                
            
                
            
        
        
        if player.position %10 == 0:
                player.score += 10
                player.position = 10
        else:
            player.position = player.position % 10
            player.score += player.position
        print("new position: {}".format(player.position))
        print("new score: {}".format(player.score))
        print("="*20)
        if (player.score >= 1000):
            break
        index = (index + 1)%len(players)
    return players, diceRoll
        
    


In [66]:
players, rollTimes = PlayTillEndDeterministic([Player(1, 4), Player(2, 8)])
print("Player with lowest score: {}".format(min(players, key=lambda x: x.score)))
player = min(players, key=lambda x: x.score)

print(player.score * rollTimes)

current position: 4
current score: 0
Player 1 rolled 1, diceRoll: 1
Player 1 rolled 2, diceRoll: 2
Player 1 rolled 3, diceRoll: 3
new position: 10
new score: 10
current position: 8
current score: 0
Player 2 rolled 4, diceRoll: 4
Player 2 rolled 5, diceRoll: 5
Player 2 rolled 6, diceRoll: 6
new position: 3
new score: 3
current position: 10
current score: 10
Player 1 rolled 7, diceRoll: 7
Player 1 rolled 8, diceRoll: 8
Player 1 rolled 9, diceRoll: 9
new position: 4
new score: 14
current position: 3
current score: 3
Player 2 rolled 10, diceRoll: 10
Player 2 rolled 11, diceRoll: 11
Player 2 rolled 12, diceRoll: 12
new position: 6
new score: 9
current position: 4
current score: 14
Player 1 rolled 13, diceRoll: 13
Player 1 rolled 14, diceRoll: 14
Player 1 rolled 15, diceRoll: 15
new position: 6
new score: 20
current position: 6
current score: 9
Player 2 rolled 16, diceRoll: 16
Player 2 rolled 17, diceRoll: 17
Player 2 rolled 18, diceRoll: 18
new position: 7
new score: 16
current position: 6


In [67]:
players, rollTimes = PlayTillEndDeterministic([Player(0, 6), Player(1, 1)])
# get the player with lowest score
print("Player with lowest score: {}".format(min(players, key=lambda x: x.score)))
player = min(players, key=lambda x: x.score)

print(player.score * rollTimes)

current position: 6
current score: 0
Player 0 rolled 1, diceRoll: 1
Player 0 rolled 2, diceRoll: 2
Player 0 rolled 3, diceRoll: 3
new position: 2
new score: 2
current position: 1
current score: 0
Player 1 rolled 4, diceRoll: 4
Player 1 rolled 5, diceRoll: 5
Player 1 rolled 6, diceRoll: 6
new position: 6
new score: 6
current position: 2
current score: 2
Player 0 rolled 7, diceRoll: 7
Player 0 rolled 8, diceRoll: 8
Player 0 rolled 9, diceRoll: 9
new position: 6
new score: 8
current position: 6
current score: 6
Player 1 rolled 10, diceRoll: 10
Player 1 rolled 11, diceRoll: 11
Player 1 rolled 12, diceRoll: 12
new position: 9
new score: 15
current position: 6
current score: 8
Player 0 rolled 13, diceRoll: 13
Player 0 rolled 14, diceRoll: 14
Player 0 rolled 15, diceRoll: 15
new position: 8
new score: 16
current position: 9
current score: 15
Player 1 rolled 16, diceRoll: 16
Player 1 rolled 17, diceRoll: 17
Player 1 rolled 18, diceRoll: 18
new position: 10
new score: 25
current position: 8
cur

In [68]:
cash = []
           # [3,4,5,6,7,8,9]
probablity = [1,3,6,7,6,3,1]

In [71]:


def PrePareNextRoll(i,player, antiplayer, depth, step):
    tmp = [Player(1, i, player.score), Player(2, antiplayer.position, antiplayer.score)]
    tmp[0].score += i
    DiracDice(tmp[1], tmp[0], depth)
    #print("tmp won: {} {}".format(tmp[0].universesWon, tmp[1].universesWon))
    player.universesWon += tmp[0].universesWon * np.uint64(probablity[step])
    antiplayer.universesWon += tmp[1].universesWon * np.uint64(probablity[step])
    #print("player won: {} {}".format(player.universesWon, antiplayer.universesWon))

def DiracDice(player, antiplayer, depth = 0):
    depth += 1
    
    cashfind = None
    if len(cash) > 0:
        cashfind = [x for x in cash if x[0] == player.score and x[1] == player.position and x[2] == antiplayer.score and x[3] == antiplayer.position]
    if cashfind is not None and len(cashfind) > 0:
        #print("found cash: {}".format(cashfind))
        player.universesWon += cashfind[0][4][0]
        antiplayer.universesWon += cashfind[0][4][1]
    else:
        step = 0
        a = (player.position+3)%10
        if a == 0:
            a = 10
        b = (player.position+9)%10
        if b == 0:
            b = 10
        if b < a:
            for i in range(a, 11):
                if (player.score + i>=21):
                    player.universesWon += np.uint64(probablity[step])
                else:
                    PrePareNextRoll(i, player, antiplayer, depth, step)
                    
                step +=1
            for i in range(1, b+1):
                if (player.score + i>=21):
                    player.universesWon += np.uint64(probablity[step])
                else:
                    
                    PrePareNextRoll(i, player, antiplayer, depth, step)
                step +=1
        else:
            for i in range(a, b+1):
                if (player.score + i>=21):
                    player.universesWon += np.uint64(probablity[step])
                else:
                    
                    PrePareNextRoll(i, player, antiplayer, depth, step)
                step +=1
        
        cash.append([player.score, player.position, antiplayer.score, antiplayer.position,  [player.universesWon, antiplayer.universesWon]])
        #print("cash: {}".format(cash[-1]))    

In [74]:
player1= Player(1, 4)
player2= Player(2, 8)
DiracDice(player1, player2)
print(player1.universesWon)
print(player2.universesWon)

In [76]:
player1 = Player(1, 6)
player2= Player(2, 1)
DiracDice(player1, player2)
print(player1.universesWon)
print(player2.universesWon)

153087536629019
175731756652760
